In [128]:
def preprocess_string(s):
    plaintext = ''
    for i in s:
        temp = ord(i)
        if (temp < 123 and temp > 96 or temp < 91 and temp > 64):
            plaintext += i
    return plaintext

### 1.

In [129]:
def create_loc_data(key):
    loc = {}
    for i in range(len(key)):
        for j in range(len(key[i])):
            for l in key[i][j]:
                loc[l] = (i, j)
    return loc
            
def playfair(s, key, bogus = 'X'):
    s = s.upper()
    pairs = []
    i = 0
    l = len(s)
    while(i < l):
        if (s[i] == s[i + 1]):
            pairs.append(s[i] + bogus)
            i += 1
        else:
            pairs.append(s[i] + s[i + 1])
            i += 2
        if (i + 2 > l):
            s = s + bogus
    
    loc = create_loc_data(key)
    ciphertext = ''
    for i, j in pairs:
        
        loc_i = loc[i]
        loc_j = loc[j]
        
        if (loc_i[0] == loc_j[0]):
            ciphertext += key[loc_i[0]][(loc_i[1] + 1) % 5][0]
            ciphertext += key[loc_j[0]][(loc_j[1] + 1) % 5][0]
        
        elif (loc_i[1] == loc_j[1]):
            ciphertext += key[(loc_i[0] + 1) % 5][loc_j[1]][0]
            ciphertext += key[(loc_j[0] + 1) % 5][loc_j[1]][0]
        
        else:
            ciphertext += key[loc_i[0]][loc_j[1]][0]
            ciphertext += key[loc_j[0]][loc_i[1]][0]
    
    return ciphertext

In [130]:
key = [['L', 'G', 'D', 'B', 'A'],
       ['Q', 'M', 'H', 'E', 'C'],
       ['U', 'R', 'N', 'IJ', 'F'],
       ['X', 'V', 'S', 'O', 'K'],
       ['Z', 'Y', 'W', 'T', 'P']]

In [131]:
s = input("Input for playfair encoder: ")
plaintext = preprocess_string(s)
print(playfair(plaintext, key))

Input for playfair encoder: we are discovered, save yourself
THGFHBNOEKOMIMHWGKMTXINVQBUK


In [127]:
'THGFHBNOEKOMIMHWGKMTXINVQBUK' == 'THGFHBNOEKOMIMHWGKMTXINVQBUK'

True

### 2. 

In [172]:
def string_to_key(s, leave = 'J', dual_with = 'I'):
    s = s + 'abcdefghijklmnopqrstuvwxyz'
    s = s.upper()
    l = [0] * 26;
    key_string = ''
    for i in s:
        if (l[ord(i) - 65] or i == leave):
            continue;
        else:
            key_string += i
            l[ord(i) - 65] = 1
            
    key = []
    idx = 0
    for i in range(5):
        row = []
        for j in range(5):
            if (key_string[idx] == dual_with):
                row.append(dual_with + leave)
            else:
                row.append(key_string[idx])
            idx += 1
        key.append(row)
    
    return key

In [173]:
s = input("Input for creating key: ") 
leave = input("Which alphabet to leave out: ")
dual_with = input("With which missing alphabet forms pair: ")
key = string_to_key(s, leave, dual_with)
print(key)

Input for creating key: commonlounge
Which alphabet to leave out: Q
With which missing alphabet forms pair: P
[['C', 'O', 'M', 'N', 'L'], ['U', 'G', 'E', 'A', 'B'], ['D', 'F', 'H', 'I', 'J'], ['K', 'PQ', 'R', 'S', 'T'], ['V', 'W', 'X', 'Y', 'Z']]


In [174]:
s = input("Input for playfair encoder: ")
plaintext = preprocess_string(s)
print(playfair(plaintext, key))

Input for playfair encoder: “PL AY FA IR ME SS AG E"
TOINIGHSEHRYYIEA
